# AE 771 Exam 2

## Imported Modules

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets as widgets
import time
import math
from IPython.display import display

## Atmo Tables

In [2]:
SIAltTable=[i*1000 for i in [0,1,3,5,10,25,50,75,100,130,160,200,300,400,600,1000]]
SITempTable=[288.15,281.651,268.65,255.65,223.252,221.552,270.65,206.65,195.08,469.27,696.29,845.56,976.01,995.83,999.85,1000]
SIPressTable=[i*101325 for i in [1,0.887,0.66919,0.53313,0.26151,0.025158,0.00078735,0.000020408,0.00000031593,1.2341e-8,2.9997e-9,8.3628e-10,8.6557e-11,1.4328e-11,8.1056e-13,7.4155e-14]]
USAltTable=[i*3.28084 for i in SIAltTable]
USTempTable=[i*1.8 for i in SITempTable]
USPressTable=[i*0.020885434273039 for i in SIPressTable]

AltTable=SIAltTable
TempTable=SITempTable
PressTable=SIPressTable
    
%run RocketEquations.py


[3.4, 0.21, 0.26, 2959, 2428, 8.9, 386, 1.26]


## Imperial Body of Code

In [3]:
Imperial = widgets.Button(description="Imperial")
outputImperial = widgets.Output()

def Imp(b):
    with outputImperial:
        # Table Selection
        
        AltTable=SIAltTable
        TempTable=SITempTable
        PressTable=SIPressTable
        
        # Fuel Selection
        Prop=PropType(Oxidizer.value,Fuel.value)
        r=Prop[0]              # Mass prop ratio
        rV=Prop[1]             # Volume prop ratio
        SG=Prop[2]             # Average specific gravity
        t1=Prop[3]*1.8         # Combustion Temperature deg R
        #cst=Prop[4]*3.28084    # C Star (speed of sound / thrust coeff)
        MW=Prop[5]             # Molar Mass lbm/lb-mol
        Isp=Prop[6]            # Specific Impulse
        k=Prop[7]              # Ratio of Specific Heats
        
        # Pre-setting Data Point Arrays
        X=[]
        Y=[]
        X15=[]
        Y15=[]
        
        # Constants
        g=32.2         # ft/s^2
        R=1544         # ft-lbf/lbm-mol-degR
        VCF=0.97       # Velocity Correction
        TCF=0.98       # Thrust Correction
        
        # Calculating Required Parameters and Tabulating
        F_IDEAL=10000  # lbf
        p1=1000        # psi         
        p2=1.58        # psi
        p3=p2
        ToF=2.5*60     # seconds
       
        dwo=71.1       # Weight Density of LOX (lbf/ft^3)
        dwf=4.4        # Weight Density of LH2 (lbf/ft^3)
        F_TRUE=F_IDEAL*TCF
        p2p1=p2/p1
        
        v2IDEAL=ExhaustVUS(k,R,t1,MW,p2p1)
        v2TRUE=v2IDEAL*VCF
        
        IspIDEAL=SpecificImpulseV(v2IDEAL,g)
        IspTRUE=SpecificImpulseV(v2TRUE,g)
        
        CfIDEAL=CfIdeal(k,p2p1)
        CfTRUE=CfIDEAL*TCF
        
        AtIDEAL=F_IDEAL/(CfIDEAL*p1)
        AtTRUE=F_IDEAL/(CfTRUE*p1)
        AR=AreaRatio(k,p2p1)
        A2IDEAL=AtIDEAL*AR
        A2TRUE=AtTRUE*AR
        
        M2=1
        Res=0.0001
        ARint=AR**-1
        
        while ARint<AR:
            M2=M2+Res
            ARint=ExpansionRatio(k,M2)
            
        wdtIDEAL=TotalWdot(F_IDEAL,IspIDEAL)
        wdtTRUE=TotalWdot(F_TRUE,IspTRUE)
        wdoIDEAL=OxWdot(wdtIDEAL,r)
        wdoTRUE=OxWdot(wdtTRUE,r)
        wdfIDEAL=FuelWdot(wdtIDEAL,r)
        wdfTRUE=FuelWdot(wdtTRUE,r)
        
        wpIDEAL=wdtIDEAL*ToF
        wpTRUE=wdtTRUE*ToF
        woIDEAL=wdoIDEAL*ToF
        woTRUE=wdoTRUE*ToF
        wfIDEAL=wdfIDEAL*ToF
        wfTRUE=wdfTRUE*ToF
        
        VdoIDEAL=wdoIDEAL/dwo
        VdoTRUE=wdoTRUE/dwo
        VdfIDEAL=wdfIDEAL/dwf
        VdfTRUE=wdfTRUE/dwf
        
        VoIDEAL=VdoIDEAL*ToF
        VoTRUE=VdoTRUE*ToF
        VfIDEAL=VdfIDEAL*ToF
        VfTRUE=VdfTRUE*ToF
            

        
        # Calculating Nozzle Dimensions
        Rt=(AtTRUE/3.141592)**0.5 # in
        Re=Rt*(AR)**0.5           # in
        AngleN=35                 # Degrees
        AngleE=5
        L=0.8*((Re-Rt)/math.tan(math.radians(15)))  # SCALING FACTOR
        
        # Converging Nozzle
        for i in range (-135,-90,1):
            X.append(1.5*Rt*math.cos(math.radians(i)))
            Y.append(1.5*Rt*math.sin(math.radians(i))+1.5*Rt+Rt)
            X15.append(1.5*Rt*math.cos(math.radians(i)))
            Y15.append(1.5*Rt*math.sin(math.radians(i))+1.5*Rt+Rt)

        # Throat and Circular Divergence
        for i in range (-90,(AngleN-90),1):
            X.append(0.382*Rt*math.cos(math.radians(i)))
            Y.append(0.382*Rt*math.sin(math.radians(i))+0.382*Rt+Rt)

        for i in range (-90,(15-90),1):    
            X15.append(0.382*Rt*math.cos(math.radians(i)))
            Y15.append(0.382*Rt*math.sin(math.radians(i))+0.382*Rt+Rt)

        # Bell Divergence
        Ex=L
        Ey=Re
        Nx=0.382*Rt*math.cos(math.radians(AngleN-90))
        Ny=0.382*Rt*math.sin(math.radians(AngleN-90))+0.382*Rt+Rt
        m1=math.tan(math.radians(AngleN))
        m2=math.tan(math.radians(AngleE))
        C1=Ny-m1*Nx
        C2=Ey-m2*Ex
        Qx=(C2-C1)/(m1-m2)
        Qy=(m1*C2-m2*C1)/(m1-m2)

        for t in range (0,100,1):
            t=t/100
            X.append(((1-t)**0.5)*Nx+2*(1-t)*t*Qx+(t**2)*Ex)
            Y.append(((1-t)**0.5)*Ny+2*(1-t)*t*Qy+(t**2)*Ey)

            
        # Combustion Chamber Work
        Dt=Rt*2
        Dc=Dt*2
        Rc=Dt
        Ac=0.5*math.pi*Rc**2
        
        Lst=31.2 #in
        Vchamber=Lst*AtTRUE
        Lconv=(Dc-Dt)/2
        Vconv=(1/3)*math.pi*(Rc**2+Rc*Rt+Rt**2)*Lconv

        Vcyl=Vchamber-Vconv
        Lcyl=Vcyl/Ac
        
        Lchamber=Lcyl+Lconv
        
        CDXinit=-Lchamber
        CDX=[CDXinit]
        CDY=[Rc]
        CDXcrv=X[0]-(Rc-Y[0])
        
        while CDXinit<CDXcrv:
            CDXinit=CDXinit+0.01
            CDY.append(Dt)
            CDX.append(CDXinit)
            
        CDYinit=Rc
        
        while CDXinit<X[0]:
            CDXinit=CDXinit+0.01
            CDYinit=CDYinit-0.01
            CDY.append(CDYinit)
            CDX.append(CDXinit)
        
        # Plotting the Bell Nozzle 
        NegY=[i*-1 for i in Y]
        NegCDY=[i*-1 for i in CDY]
        plt.xlabel('Length in Inches')
        plt.ylabel('Width in Inches')
        plt.plot(X,Y,color='tab:gray')
        plt.plot(CDX,CDY,color='tab:gray')
        plt.plot(X,NegY,color='tab:gray')
        plt.plot(CDX,NegCDY,color='tab:gray')
        plt.xlim([-20,30])
        plt.ylim([-16.2,16.2])
        plt.title('Rao Nozzle Plot')
        plt.show()

        # 15 Degree Half Angle Divergence
        Nx15=0.382*Rt*math.cos(math.radians(15-90))
        Ny15=0.382*Rt*math.sin(math.radians(15-90))+0.382*Rt+Rt

        xtemp=int(round(Nx))
        while math.tan(math.radians(15))*xtemp+Ny15 < Re:
            X15.append(xtemp)
            Y15.append(math.tan(math.radians(15))*xtemp+Ny15)
            xtemp=xtemp+0.1

        # Plotting 15 degree Half Angle Nozzle
        NegY15=[i*-1 for i in Y15]
        plt.xlabel('Length in Inches')
        plt.ylabel('Width in Inches')
        plt.plot(X15,Y15,color='tab:gray')
        plt.plot(CDX,CDY,color='tab:gray')
        plt.plot(X15,NegY15,color='tab:gray')
        plt.plot(CDX,NegCDY,color='tab:gray')
        plt.xlim([-20,30])
        plt.ylim([-16.2,16.2])
        plt.title('15 Degree Angle Nozzle Plot')
        plt.show()
        
        # Combustion Instability
        RadialL=2*(Dc/12)
        TanL=2*math.pi*(Dc/12)
        AxialL=2*(Lchamber/12)
        
        a=(k*t1*R*32.2/MW)**0.5
        
        RadialF=a/RadialL
        TanF=a/TanL
        AxialF=a/AxialL
        
        print('---------------------------------------------------------')
        print('---------Velocities, Areas, Flowrates, Weights-----------')
        print('---------------------------------------------------------\n')
        
        print('Value                      Ideal         Actual')
        print('Exit Velocity:             ','{0:.0f}'.format(v2IDEAL),'(ft/s)    '
              '{0:.0f}'.format(v2TRUE),'(ft/s)\n')
        print('Throat Area:               ','{0:.3f}'.format(AtIDEAL),'(ft^2)      '
              '{0:.3f}'.format(AtTRUE),'(ft^2)\n')
        print('Exit Area:                 ','{0:.1f}'.format(A2IDEAL),'(ft^2)    '
              '{0:.1f}'.format(A2TRUE),'(ft^2)\n')
        print('Exit Mach:                 ','{0:.2f}'.format(M2),'(~)        '
              '{0:.2f}'.format(M2),'(~)\n')
        print('Total Weight Flow Rate:    ','{0:.2f}'.format(wdtIDEAL),'(lbf/s)    '
              '{0:.2f}'.format(wdtTRUE),'(lbf/s)\n')
        print('Oxidizer Weight Flow Rate: ','{0:.2f}'.format(wdoIDEAL),'(lbf/s)    '
              '{0:.2f}'.format(wdoTRUE),'(lbf/s)\n')
        print('Fuel Weight Flow Rate:     ','{0:.2f}'.format(wdfIDEAL),'(lbf/s)    '
              '{0:.2f}'.format(wdfTRUE),'(lbf/s)\n')
        print('Propellant Weight:         ','{0:.0f}'.format(wpIDEAL),'(lbf)    '
              '{0:.0f}'.format(wpTRUE),'(lbf)\n')
        print('Oxidizer Weight:           ','{0:.0f}'.format(woIDEAL),'(lbf)     '
              '{0:.0f}'.format(woTRUE),'(lbf)\n')
        print('Fuel Weight:               ','{0:.0f}'.format(wfIDEAL),'(lbf)     '
              '{0:.0f}'.format(wfTRUE),'(lbf)\n')
        print('Oxidizer Volume Flow Rate: ','{0:.2f}'.format(VdoIDEAL),'(ft^3/s)    '
              '{0:.2f}'.format(VdoTRUE),'(ft^3/s)\n')
        print('Fuel Volume Flow Rate:     ','{0:.2f}'.format(VdfIDEAL),'(ft^3/s)    '
              '{0:.2f}'.format(VdfTRUE),'(ft^3/s)\n')
        print('Oxidizer Volume:           ','{0:.1f}'.format(VoIDEAL),'(ft^3)    '
              '{0:.1f}'.format(VoTRUE),'(ft^3)\n')
        print('Fuel Volume:               ','{0:.1f}'.format(VfIDEAL),'(ft^3)    '
              '{0:.1f}'.format(VfTRUE),'(ft^3)\n')
        
        print('---------------------------------------------------------')
        print('-------------------------Frequencies---------------------')
        print('---------------------------------------------------------\n')
        print('L* (Assumed):',Lst,'(in)')
        print('Convergence Angle (Assumed): 45 (deg)')
        print('Converging Duct Length (Calculated):','{0:.1f}'.format(-CDXcrv),'(in)')
        print('Cylindrical Duct Length (Calculated):',Lcyl,'(in)')
        print('Chamber Diameter (Calculated):','{0:.1f}'.format(Dc),'(in)\n')
        
        print('Radial Resonance Frequency:','{0:.2f}'.format(RadialF),'(Hz)')
        print('Tangential Resonance Frequency:','{0:.2f}'.format(TanF),'(Hz)')
        print('Axial Resonance Frequency:','{0:.2f}'.format(AxialF),'(Hz)')
        
        
Imperial.on_click(Imp)

## Metric

In [4]:
Metric = widgets.Button(description="Metric")
outputMetric = widgets.Output()

def SI(b):
    with outputMetric:
        # Table Selection
        
        AltTable=SIAltTable
        TempTable=SITempTable
        PressTable=SIPressTable
        
        # Fuel Selection
        Prop=PropType(Oxidizer.value,Fuel.value)
        r=Prop[0]      # Mass prop ratio
        rV=Prop[1]     # Volume prop ratio
        SG=Prop[2]     # Average specific gravity
        t1=Prop[3]     # Combustion Temperature K
        #cst=Prop[4]    # C Star (speed of sound / thrust coeff)
        MW=Prop[5]     # Molar Mass kg/mol
        Isp=Prop[6]    #Specific Impulse
        k=Prop[7]      # Ratio of Specific Heats
        
        # Pre-setting Data Point Arrays
        X=[]
        Y=[]
        X15=[]
        Y15=[]
        
        # Constants
        g=9.81         # m/s^2
        R=8314         # kJ/mol-K
        VCF=0.97       # Velocity Correction
        TCF=0.98       # Thrust Correction
        
        # Calculating Required Parameters and Tabulating
        F_IDEAL=44482  # N
        p1=6894757     # Pa
        p2=10893       # Pa
        p3=p2
        ToF=2.5*60     # seconds
                
        dwo=11175.6       # Weight Density of LOX (lbf/ft^3)
        dwf=691.6         # Weight Density of LH2 (lbf/ft^3)
        F_TRUE=F_IDEAL*TCF
        p2p1=p2/p1
        
        v2IDEAL=ExhaustVMol(k,R,t1,MW,p2p1)
        v2TRUE=v2IDEAL*VCF
        
        IspIDEAL=SpecificImpulseV(v2IDEAL,g)
        IspTRUE=SpecificImpulseV(v2TRUE,g)
        
        CfIDEAL=CfIdeal(k,p2p1)
        CfTRUE=CfIDEAL*TCF
        
        AtIDEAL=F_IDEAL/(CfIDEAL*p1)
        AtTRUE=F_IDEAL/(CfTRUE*p1)
        AR=AreaRatio(k,p2p1)
        A2IDEAL=AtIDEAL*AR
        A2TRUE=AtTRUE*AR
        
        M2=1
        Res=0.0001
        ARint=AR**-1
        
        while ARint<AR:
            M2=M2+Res
            ARint=ExpansionRatio(k,M2)
            
        wdtIDEAL=TotalWdot(F_IDEAL,IspIDEAL)
        wdtTRUE=TotalWdot(F_TRUE,IspTRUE)
        wdoIDEAL=OxWdot(wdtIDEAL,r)
        wdoTRUE=OxWdot(wdtTRUE,r)
        wdfIDEAL=FuelWdot(wdtIDEAL,r)
        wdfTRUE=FuelWdot(wdtTRUE,r)
        
        wpIDEAL=wdtIDEAL*ToF
        wpTRUE=wdtTRUE*ToF
        woIDEAL=wdoIDEAL*ToF
        woTRUE=wdoTRUE*ToF
        wfIDEAL=wdfIDEAL*ToF
        wfTRUE=wdfTRUE*ToF
        
        VdoIDEAL=wdoIDEAL/dwo
        VdoTRUE=wdoTRUE/dwo
        VdfIDEAL=wdfIDEAL/dwf
        VdfTRUE=wdfTRUE/dwf
        
        VoIDEAL=VdoIDEAL*ToF
        VoTRUE=VdoTRUE*ToF
        VfIDEAL=VdfIDEAL*ToF
        VfTRUE=VdfTRUE*ToF
            

        
        # Calculating Nozzle Dimensions
        Rt=(AtTRUE/3.141592)**0.5 # in
        Re=Rt*(AR)**0.5           # in
        AngleN=35                 # Degrees
        AngleE=5
        L=0.8*((Re-Rt)/math.tan(math.radians(15))) # SCALING FACTOR
        
        # Converging Nozzle
        for i in range (-135,-90,1):
            X.append(1.5*Rt*math.cos(math.radians(i)))
            Y.append(1.5*Rt*math.sin(math.radians(i))+1.5*Rt+Rt)
            X15.append(1.5*Rt*math.cos(math.radians(i)))
            Y15.append(1.5*Rt*math.sin(math.radians(i))+1.5*Rt+Rt)

        # Throat and Circular Divergence
        for i in range (-90,(AngleN-90),1):
            X.append(0.382*Rt*math.cos(math.radians(i)))
            Y.append(0.382*Rt*math.sin(math.radians(i))+0.382*Rt+Rt)

        for i in range (-90,(15-90),1):    
            X15.append(0.382*Rt*math.cos(math.radians(i)))
            Y15.append(0.382*Rt*math.sin(math.radians(i))+0.382*Rt+Rt)

        # Bell Divergence
        Ex=L
        Ey=Re
        Nx=0.382*Rt*math.cos(math.radians(AngleN-90))
        Ny=0.382*Rt*math.sin(math.radians(AngleN-90))+0.382*Rt+Rt
        m1=math.tan(math.radians(AngleN))
        m2=math.tan(math.radians(AngleE))
        C1=Ny-m1*Nx
        C2=Ey-m2*Ex
        Qx=(C2-C1)/(m1-m2)
        Qy=(m1*C2-m2*C1)/(m1-m2)

        for t in range (0,100,1):
            t=t/100
            X.append(((1-t)**0.5)*Nx+2*(1-t)*t*Qx+(t**2)*Ex)
            Y.append(((1-t)**0.5)*Ny+2*(1-t)*t*Qy+(t**2)*Ey)

            
        # Combustion Chamber Work
        Dt=Rt*2
        Dc=Dt*2
        Rc=Dt
        Ac=0.5*math.pi*Rc**2
        
        Lst=0.8 #in
        Vchamber=Lst*AtTRUE
        Lconv=(Dc-Dt)/2
        Vconv=(1/3)*math.pi*(Rc**2+Rc*Rt+Rt**2)*Lconv
        
        Vcyl=Vchamber-Vconv
        Lcyl=Vcyl/Ac
        
        Lchamber=Lcyl+Lconv
        
        CDXinit=-Lchamber
        CDX=[CDXinit]
        CDY=[Rc]
        CDXcrv=X[0]-(Rc-Y[0])
        
        while CDXinit<CDXcrv:
            CDXinit=CDXinit+0.01
            CDY.append(Dt)
            CDX.append(CDXinit)
            
        CDYinit=Rc
        
        while CDXinit<X[0]:
            CDXinit=CDXinit+0.01
            CDYinit=CDYinit-0.01
            CDY.append(CDYinit)
            CDX.append(CDXinit)
        
        # Plotting the Bell Nozzle 
        NegY=[i*-1 for i in Y]
        NegCDY=[i*-1 for i in CDY]
        plt.xlabel('Length in Inches')
        plt.ylabel('Width in Inches')
        plt.plot(X,Y,color='tab:gray')
        plt.plot(CDX,CDY,color='tab:gray')
        plt.plot(X,NegY,color='tab:gray')
        plt.plot(CDX,NegCDY,color='tab:gray')
        plt.xlim([-0.508,0.762])
        plt.ylim([-0.41,0.41])
        plt.title('Rao Nozzle Plot')
        plt.show()

        # 15 Degree Half Angle Divergence
        Nx15=0.382*Rt*math.cos(math.radians(15-90))
        Ny15=0.382*Rt*math.sin(math.radians(15-90))+0.382*Rt+Rt

        xtemp=int(round(Nx))
        while math.tan(math.radians(15))*xtemp+Ny15 < Re:
            X15.append(xtemp)
            Y15.append(math.tan(math.radians(15))*xtemp+Ny15)
            xtemp=xtemp+0.1

        # Plotting 15 degree Half Angle Nozzle
        NegY15=[i*-1 for i in Y15]
        plt.xlabel('Length in Inches')
        plt.ylabel('Width in Inches')
        plt.plot(X15,Y15,color='tab:gray')
        plt.plot(CDX,CDY,color='tab:gray')
        plt.plot(X15,NegY15,color='tab:gray')
        plt.plot(CDX,NegCDY,color='tab:gray')
        plt.xlim([-0.508,0.762])
        plt.ylim([-0.41,0.41])
        plt.title('15 Degree Angle Nozzle Plot')
        plt.show()
        
        # Combustion Instability
        RadialL=2*(Dc/12)
        TanL=2*math.pi*(Dc/12)
        AxialL=2*(Lchamber/12)
        
        a=(k*t1*R/MW)**0.5
        
        RadialF=a/RadialL
        TanF=a/TanL
        AxialF=a/AxialL
        
        print('---------------------------------------------------------')
        print('---------Velocities, Areas, Flowrates, Weights-----------')
        print('---------------------------------------------------------\n')
        
        print('Value                      Ideal         Actual')
        print('Exit Velocity:             ','{0:.0f}'.format(v2IDEAL),'(m/s)    '
              '{0:.0f}'.format(v2TRUE),'(m/s)\n')
        print('Throat Area:               ','{0:.3f}'.format(AtIDEAL),'(m^2)      '
              '{0:.3f}'.format(AtTRUE),'(m^2)\n')
        print('Exit Area:                 ','{0:.1f}'.format(A2IDEAL),'(m^2)    '
              '{0:.1f}'.format(A2TRUE),'(m^2)\n')
        print('Exit Mach:                 ','{0:.2f}'.format(M2),'(~)        '
              '{0:.2f}'.format(M2),'(~)\n')
        print('Total Weight Flow Rate:    ','{0:.2f}'.format(wdtIDEAL),'(N/s)    '
              '{0:.2f}'.format(wdtTRUE),'(N/s)\n')
        print('Oxidizer Weight Flow Rate: ','{0:.2f}'.format(wdoIDEAL),'(N/s)    '
              '{0:.2f}'.format(wdoTRUE),'(N/s)\n')
        print('Fuel Weight Flow Rate:     ','{0:.2f}'.format(wdfIDEAL),'(N/s)    '
              '{0:.2f}'.format(wdfTRUE),'(N/s)\n')
        print('Propellant Weight:         ','{0:.0f}'.format(wpIDEAL),'(N)    '
              '{0:.0f}'.format(wpTRUE),'(N)\n')
        print('Oxidizer Weight:           ','{0:.0f}'.format(woIDEAL),'(N)     '
              '{0:.0f}'.format(woTRUE),'(N)\n')
        print('Fuel Weight:               ','{0:.0f}'.format(wfIDEAL),'(N)     '
              '{0:.0f}'.format(wfTRUE),'(N)\n')
        print('Oxidizer Volume Flow Rate: ','{0:.3f}'.format(VdoIDEAL),'(m^3/s)    '
              '{0:.3f}'.format(VdoTRUE),'(m^3/s)\n')
        print('Fuel Volume Flow Rate:     ','{0:.3f}'.format(VdfIDEAL),'(m^3/s)    '
              '{0:.3f}'.format(VdfTRUE),'(m^3/s)\n')
        print('Oxidizer Volume:           ','{0:.3f}'.format(VoIDEAL),'(m^3)    '
              '{0:.3f}'.format(VoTRUE),'(m^3)\n')
        print('Fuel Volume:               ','{0:.3f}'.format(VfIDEAL),'(m^3)    '
              '{0:.3f}'.format(VfTRUE),'(m^3)\n')
        
        print('---------------------------------------------------------')
        print('-------------------------Frequencies---------------------')
        print('---------------------------------------------------------\n')
        
        print('L* (Assumed):',Lst,'(m)')
        print('Convergence Angle (Assumed): 45 (deg)')
        print('Converging Duct Length (Calculated):','{0:.1f}'.format(-CDXcrv),'(m)')
        print('Cylindrical Duct Length (Calculated):',Lcyl,'(m)')
        print('Chamber Diameter (Calculated):','{0:.1f}'.format(Dc),'(m)\n')
        
        print('Radial Resonance Frequency:','{0:.2f}'.format(RadialF),'(Hz)')
        print('Tangential Resonance Frequency:','{0:.2f}'.format(TanF),'(Hz)')
        print('Axial Resonance Frequency:','{0:.2f}'.format(AxialF),'(Hz)')
        
Metric.on_click(SI)

## GUI

In [5]:
Oxidizer=widgets.Dropdown(
    options=['Oxygen','Flourine','Nitrogen Tetroxide','Nitric Acid','Hydrogen Peroxide'],
    value='Oxygen',
    description='Oxidizer:',
    disabled=False,
)

Fuel=widgets.Dropdown(
    options=['Methane','Hydrazine','Hydrogen','RP-1','UDMH','UDMH/Hydrazine','MMH'],
    value='Hydrogen',
    description='Fuel:',
    disabled=False,
)

DispWindows1=widgets.HBox([Oxidizer,Fuel],
                        layout=widgets.Layout(overflow='visible',
                        align_items='center',
                        align_contents='center',)
)

display(DispWindows1)
display(Imperial, outputImperial)
display(Metric, outputMetric)

Button(description='Imperial', style=ButtonStyle())

Output()

Button(description='Metric', style=ButtonStyle())

Output()